# Introduction

## Key Concepts

### Spark
Apache Spark is a unified computing engine and a set of libraries for parallel data processing on computer clusters. Spark supports multiple widely used programming languages (Python, Java, Scala, and R), includes libraries for diverse tasks ranging from SQL to streaming and machine learning, and runs anywehere from a laoptop to a cluster of thousands of servers. 

### Spark Application
Spark Applications consist of a driver process and a set of executor processes. 


### Driver
The driver process runs your main() function, sits on a node in the cluster, and is responsible for maintaining infomration about the Spark Application, responding to a user's program or input; and anlyzing, distributing, and scheduling work across the executors. 

### Executor
The executors are responsible for actually carrying out the work that the driver assigns them. Each executor is responsible for exeucting code assigned ot it by the driver, and reporting the state of the computation on that executor back to the driver node. 

### Cluster Manager
The cluster manager controls physical machines and allocates resources to Spark Applications. This can be one of three core cluster managers: Spark's standalone cluster manager, YARN, or Mesos. 

### Partition
A partition is a collection of rows that sits on one physical machine in your cluster.

### Execution Modes
* Cluster mode. In a cluster mode, a user submits a pre-complied JAR, Python or R script to a cluster manager. The cluster manager then launches the driver process on a worker node inside the cluster, in addition to the executor processes. The cluster manager is responsible for maintaining all Spark Application-related processes. 
* Client mode. Client mode is nearly the same as cluster mode except that the Spark driver remains on the client machine that submitted the application. This means that the client machine is responsible for maintaining the Spark driver process, and the cluster manager maintains the executor processses.
* Local mode. It runs the entire Spark Application on a single machine. It achieves parallelism through threads on that single machine.

### A Spark Job
In general, there should be one Spark job for one action. Actions always return results. Each job breaks down into a series of stages, the number of which depends on how many shuffle operations need to take place.


### Stages
Stages in Spark represent groups of tasks that can be executed together to compute the same operation on multiple machines. In general, Spark will try to pack as much work as possible (i.e., as many transformations as possible inside your job) into the same stage, but the engine starts new stages after operations called shuffles. A shuffle represents a physical repartitioning of the data—for example, sorting a DataFrame, or grouping data that was loaded from a file by key (which requires sending records with the same key to the same node). This type of repartitioning requires coordinating across executors to move data around. Spark starts a new stage after each shuffle, and keeps track of what order the stages must run in to compute the final result.

### Tasks
Stages in Spark consist of tasks. Each task corresponds to a combination of blocks of data and a set of transformations that will run on a single executor. If there is one big partition in our dataset, we will have one task. If there are 1,000 little partitions, we will have 1,000 tasks that can be executed in parallel. A task is just a unit of computation applied to a unit of data (the partition).


### Distribution of Executors, Cores and Memory for a Spark Application
Theory:
* Hadoop/Yarn/OS Deamons: When we run spark application using a cluster manager like Yarn, there’ll be several daemons that’ll run in the background. So, while specifying num-executors, we need to make sure that we leave aside enough cores (~1 core per node) for these daemons to run smoothly.
* Yarn ApplicationMaster (AM): ApplicationMaster is responsible for negotiating resources from the ResourceManager and working with the NodeManagers to execute and monitor the containers and their resource consumption. If we are running spark on yarn, then we need to budget in the resources that AM would need (~1024MB and 1 Executor).
* HDFS Throughput: HDFS client has trouble with tons of concurrent threads. It was observed that HDFS achieves full write throughput with ~5 tasks per executor. So it’s good to keep the number of cores per executor below that number. * Spark Yarn executor MemoryOverhead which is maxima of 384MB or 7% of the executor memory.

Example: 
Consider a 10 node cluster with 16 cores and 64GB RAM per node.     
    * Based on the recommendations mentioned above for good HDFS throughput, Let’s assign --executor-core = 5
    * Leave 1 core per node for Hadoop/Yarn daemons => Num cores available per node = 16-1 = 15
    * Total available of cores in cluster = 15 x 10 = 150
    * Number of available executors = (total cores/num-cores-per-executor) = 150 / 5 = 30
    * Leaving 1 executor for ApplicationManager => --num-executors = 29
    * Number of executors per node = 30/10 = 3
    * Memory per executor = 64GB/3 = 21GB
    * Counting off heap overhead = 7% of 21GB = 3GB
    * Actual --executor-memory = 21 - 3 = 18GB

### Transformation
A transformation instructs Spark how you would lije to modify a DataFrame to do what you want. There are two types of transformations: Narrow dependencies are those for which each input partition will contribute to only one output partition; a wide dependency style transformation will have input partitions contributing to many output partitions. With narrow transformations, Spark will automatically perform an operation called pipelining, meaning that if we specify multiple filters on DataFrames, they will all be performed in memory. For shuffles, Spark writes results to disk.

### Lazy Evaluation
Spark waits until the very last moment to execute the graph of computation instructions. This provides immense benefits because Spark can optimize the entire data flow from end to end. 


### Action
An action instructs Spark to compute a result from a series of transformations. There are three kinds of actuibs:
* Actions to view data in the console
* Actions to collect data to native objects in teh respective language
* Actions to write to output data 

### DataFrames and Datasets
DataFrames and Datasets are distributed table-like collections with well-defined rows and columns. They represent immutable, lazily evaluated plans that specify what operations to apply to data residing at a location to generate some output. When we perform an action on a DataFrame, we instruct Spark to perform the actual transformations and return the result. These represent plans of how to manipulate rows and columns to compute the user's desired result.

The difference between DataFrames and Datasets lies in "types". Both have types though. For DataFrames, Spark maintains types completelt and only checks whether those types line up to those specified in the schema at runtime. Datasets, on the other hand, check whether types conform to the specificiation at complie time. Datasets are only available to JVM-based languages (Scala and Java). 

## Getting Started

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local").appName("Hello World").getOrCreate()

In [ ]:
myRange = spark.range(1000).toDF("number")

In [ ]:
divisBy2 = myRange.where("number % 2 = 0")

In [ ]:
flightData2015 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("../data/flight-data/csv/2015-summary.csv")

In [ ]:
flightData2015.take(3)

In [ ]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [ ]:
flightData2015.count()

In [ ]:
sqlWay = spark.sql("""
    SELECT DEST_COUNTRY_NAME, count(1)
    FROM flight_data_2015
    GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .count()

sqlWay.explain()
dataFrameWay.explain()

In [ ]:
flightData2015.select(max("count")).take(1)

In [ ]:
maxSql = spark.sql("""
    SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
    FROM flight_data_2015
    GROUP BY DEST_COUNTRY_NAME
    ORDER BY sum(count) DESC
    LIMIT 5
""")

maxSql.show()

In [ ]:
flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .show()

In [ ]:
flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .take(5)